In [20]:
import pandas as pd

In [21]:
df = pd.read_excel("Fast Food Analysis Project Spreadsheet.xlsx")

In [22]:
df.head()

,Unnamed: 0,Restaurant,Item Name,Calories,Protein (g),Total Fats (g),Sodium (mg),Total Sugars (g),Cholestrol (mg),Price ($),Carbs (g),Fiber (g)
0,NaN,McDonald's,"Bacon, Egg, and Cheese McGriddle",430,17,21,1230,15,215,5.79,44,2
1,NaN,McDonald's,"Sausage, Egg, and Cheese McGriddle",550,19,33,1290,15,235,5.99,44,2
2,NaN,McDonald's,Quarter Pounder with Cheese,520,30,26,1140,10,95,6.89,42,2
3,NaN,McDonald's,Small Fry,230,3,11,190,0,0,3.49,31,3
4,NaN,McDonald's,4 pc. Chicken McNuggets,170,9,10,340,0,25,3.69,10,0


In [23]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [24]:
df.head()

,Restaurant,Item Name,Calories,Protein (g),Total Fats (g),Sodium (mg),Total Sugars (g),Cholestrol (mg),Price ($),Carbs (g),Fiber (g)
0,McDonald's,"Bacon, Egg, and Cheese McGriddle",430,17,21,1230,15,215,5.79,44,2
1,McDonald's,"Sausage, Egg, and Cheese McGriddle",550,19,33,1290,15,235,5.99,44,2
2,McDonald's,Quarter Pounder with Cheese,520,30,26,1140,10,95,6.89,42,2
3,McDonald's,Small Fry,230,3,11,190,0,0,3.49,31,3
4,McDonald's,4 pc. Chicken McNuggets,170,9,10,340,0,25,3.69,10,0


In [25]:
df.shape

(49, 11)

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Restaurant        49 non-null     object 
 1   Item Name         49 non-null     object 
 2   Calories          49 non-null     int64  
 3   Protein (g)       49 non-null     int64  
 4   Total Fats (g)    49 non-null     int64  
 5   Sodium (mg)       49 non-null     int64  
 6   Total Sugars (g)  49 non-null     int64  
 7   Cholestrol (mg)   49 non-null     int64  
 8   Price ($)         49 non-null     float64
 9   Carbs (g)         49 non-null     int64  
 10  Fiber (g)         49 non-null     int64  
dtypes: float64(1), int64(8), object(2)
memory usage: 4.3+ KB


In [27]:
print(df.columns)

Index(['Restaurant', 'Item Name', 'Calories ', 'Protein (g)', 'Total Fats (g)',
       'Sodium (mg)', 'Total Sugars (g)', 'Cholestrol (mg)', 'Price ($)',
       'Carbs (g)', 'Fiber (g)'],
      dtype='object')


In [28]:
df = df.rename(columns={'Restaurant': 'restaurant_name', 'Item Name': 'item_name', 'Calories ': 'calories', 'Protein (g)': 'protein_g',
 'Total Fats (g)': 'fats_g', 'Sodium (mg)': 'sodium_mg', 'Total Sugars (g)': 'sugars_g',
 'Cholestrol (mg)': 'cholestrol_mg', 'Price ($)': 'price', 'Carbs (g)': 'carbs_g', 
 'Fiber (g)': 'fiber_g'})
df.head()

,restaurant_name,item_name,calories,protein_g,fats_g,sodium_mg,sugars_g,cholestrol_mg,price,carbs_g,fiber_g
0,McDonald's,"Bacon, Egg, and Cheese McGriddle",430,17,21,1230,15,215,5.79,44,2
1,McDonald's,"Sausage, Egg, and Cheese McGriddle",550,19,33,1290,15,235,5.99,44,2
2,McDonald's,Quarter Pounder with Cheese,520,30,26,1140,10,95,6.89,42,2
3,McDonald's,Small Fry,230,3,11,190,0,0,3.49,31,3
4,McDonald's,4 pc. Chicken McNuggets,170,9,10,340,0,25,3.69,10,0


In [29]:
df.head()

,restaurant_name,item_name,calories,protein_g,fats_g,sodium_mg,sugars_g,cholestrol_mg,price,carbs_g,fiber_g
0,McDonald's,"Bacon, Egg, and Cheese McGriddle",430,17,21,1230,15,215,5.79,44,2
1,McDonald's,"Sausage, Egg, and Cheese McGriddle",550,19,33,1290,15,235,5.99,44,2
2,McDonald's,Quarter Pounder with Cheese,520,30,26,1140,10,95,6.89,42,2
3,McDonald's,Small Fry,230,3,11,190,0,0,3.49,31,3
4,McDonald's,4 pc. Chicken McNuggets,170,9,10,340,0,25,3.69,10,0


In [30]:
df.head()

,restaurant_name,item_name,calories,protein_g,fats_g,sodium_mg,sugars_g,cholestrol_mg,price,carbs_g,fiber_g
0,McDonald's,"Bacon, Egg, and Cheese McGriddle",430,17,21,1230,15,215,5.79,44,2
1,McDonald's,"Sausage, Egg, and Cheese McGriddle",550,19,33,1290,15,235,5.99,44,2
2,McDonald's,Quarter Pounder with Cheese,520,30,26,1140,10,95,6.89,42,2
3,McDonald's,Small Fry,230,3,11,190,0,0,3.49,31,3
4,McDonald's,4 pc. Chicken McNuggets,170,9,10,340,0,25,3.69,10,0


In [31]:
df.isnull().sum()


restaurant_name    0
item_name          0
calories           0
protein_g          0
fats_g             0
sodium_mg          0
sugars_g           0
cholestrol_mg      0
price              0
carbs_g            0
fiber_g            0
dtype: int64

In [32]:
df.duplicated().sum()

np.int64(0)

In [33]:
## starting tier one analysis: basic averages, ranks of each restaurant and item with the highest calories, protein etc. 
df.describe()

,calories,protein_g,fats_g,sodium_mg,sugars_g,cholestrol_mg,price,carbs_g,fiber_g
count,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000,49.000000
mean,444.693878,20.204082,22.979592,1018.469388,8.959184,68.265306,6.604286,40.306122,2.938776
std,226.265902,14.850617,14.138437,733.132693,16.724432,64.835939,2.719396,21.897683,3.044176
min,80.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.820000,10.000000,0.000000
25%,270.000000,7.000000,14.000000,380.000000,2.000000,20.000000,4.550000,26.000000,1.000000
50%,430.000000,18.000000,21.000000,1010.000000,5.000000,60.000000,6.330000,42.000000,2.000000
75%,570.000000,29.000000,31.000000,1420.000000,10.000000,90.000000,7.490000,46.000000,4.000000
max,1270.000000,61.000000,78.000000,3070.000000,113.000000,315.000000,13.980000,129.000000,14.000000


In [47]:
## gonna use a loop to find top and bottom 10 of each metric (protein, fiber etc)
metrics = ['calories', 'protein_g', 'fats_g', 'sodium_mg', 'cholestrol_mg', 'price', 'carbs_g', 'fiber_g']

for metric in metrics:
    print(f"Top 10 {metric.title()}")
    display(df.sort_values(by=metric, ascending=False).head(10))

    print(f"Bottom 10 {metric.title()}")
    display(df.sort_values(by=metric, ascending=False).tail(10))

Top 10 Calories


,restaurant_name,item_name,calories,protein_g,fats_g,sodium_mg,sugars_g,cholestrol_mg,price,carbs_g,fiber_g
40,Zaxbys,Chicken Bacon Ranch Loaded Fries,1270,61,78,3070,2,145,12.39,88,8
43,Zaxbys,The Fried Cobb Zalad,780,55,45,2130,5,315,10.55,43,4
32,Taco Bell,Nachos BellGrande (Beef),730,17,39,1180,4,20,8.16,80,13
46,Zaxbys,Birthday Cake Milkshake,730,13,19,600,113,60,3.30,129,0
34,Taco Bell,Grilled Cheese Burrito,710,25,39,1510,4,75,6.82,64,6
39,Zaxbys,Fried White Cheddar Bites,700,27,46,1550,2,75,7.11,46,3
42,Zaxbys,Kickin Chicken Sandwich,690,34,39,2760,5,65,10.02,51,2
17,Chick-fil-A,Smokehouse BBQ Bacon Sandwich w/ Reg Filet,670,41,31,1930,12,110,10.45,57,3
45,Zaxbys,The Fried House Zalad,660,44,37,1920,4,90,13.98,42,4
16,Chick-fil-A,Cool Wrap,660,43,45,1420,5,110,11.25,32,14


Bottom 10 Calories


,restaurant_name,item_name,calories,protein_g,fats_g,sodium_mg,sugars_g,cholestrol_mg,price,carbs_g,fiber_g
3,McDonald's,Small Fry,230,3,11,190,0,0,3.49,31,3
27,Starbucks,Tall Caffé Mocha,200,7,9,75,18,30,6.15,22,2
33,Taco Bell,Nacho Cheese Doritos Locos Tacos Supreme,190,8,11,380,1,30,4.62,15,3
4,McDonald's,4 pc. Chicken McNuggets,170,9,10,340,0,25,3.69,10,0
36,Taco Bell,Cinnamon Twists,170,1,6,115,10,0,1.82,27,1
26,Starbucks,Birthday Cake Pop,160,2,8,95,16,10,3.25,21,0
19,Starbucks,Small Blonde Vanilla Latte,130,6,3,0,17,15,6.55,18,0
28,Starbucks,Short Caramel Macchiato,120,5,4,80,15,15,6.15,16,0
23,Starbucks,Tall Strawberry Açaí Lemonade Refresher,80,0,0,10,16,0,5.95,18,1
24,Starbucks,Tall Iced Shaken Espresso,80,3,1,45,11,5,5.15,13,0


Top 10 Protein_G


...

,restaurant_name,item_name,calories,protein_g,fats_g,sodium_mg,sugars_g,cholestrol_mg,price,carbs_g,fiber_g
23,Starbucks,Tall Strawberry Açaí Lemonade Refresher,80,0,0,10,16,0,5.95,18,1
19,Starbucks,Small Blonde Vanilla Latte,130,6,3,0,17,15,6.55,18,0
11,Chick-fil-A,Small Mac & Cheese,270,12,17,710,2,40,4.55,17,2
48,Zaxbys,5 Chicken Fingerz,420,45,20,2030,0,90,11.34,16,0
28,Starbucks,Short Caramel Macchiato,120,5,4,80,15,15,6.15,16,0
12,Chick-fil-A,3 ct. Chick-n-Strips,310,29,14,870,2,75,7.49,16,0
33,Taco Bell,Nacho Cheese Doritos Locos Tacos Supreme,190,8,11,380,1,30,4.62,15,3
24,Starbucks,Tall Iced Shaken Espresso,80,3,1,45,11,5,5.15,13,0
14,Chick-fil-A,8 ct. Nuggets,250,27,11,1210,1,85,7.15,11,0
4,McDonald's,4 pc. Chicken McNuggets,170,9,10,340,0,25,3.69,10,0


Top 10 Fiber_G


,restaurant_name,item_name,calories,protein_g,fats_g,sodium_mg,sugars_g,cholestrol_mg,price,carbs_g,fiber_g
16,Chick-fil-A,Cool Wrap,660,43,45,1420,5,110,11.25,32,14
32,Taco Bell,Nachos BellGrande (Beef),730,17,39,1180,4,20,8.16,80,13
37,Taco Bell,Mexican Pizza,530,19,28,1010,3,40,7.31,50,8
40,Zaxbys,Chicken Bacon Ranch Loaded Fries,1270,61,78,3070,2,145,12.39,88,8
31,Taco Bell,Beefy 5-Layer Burrito,490,17,18,1290,4,30,5.11,65,7
30,Taco Bell,Crunchwrap Supreme,530,15,20,1220,6,20,7.31,73,6
34,Taco Bell,Grilled Cheese Burrito,710,25,39,1510,4,75,6.82,64,6
13,Chick-fil-A,Medium Waffle Potato Fries,420,5,24,240,1,0,3.59,45,5
44,Zaxbys,Regular Crinkle Fries,330,5,14,580,0,0,3.81,47,5
38,Taco Bell,Cheesy Gordita Crunch,490,20,28,850,4,55,6.33,42,5


Bottom 10 Fiber_G


,restaurant_name,item_name,calories,protein_g,fats_g,sodium_mg,sugars_g,cholestrol_mg,price,carbs_g,fiber_g
20,Starbucks,"Sausage, Cheddar, & Egg Sandwich",480,18,29,890,2,165,5.95,34,1
46,Zaxbys,Birthday Cake Milkshake,730,13,19,600,113,60,3.30,129,0
24,Starbucks,Tall Iced Shaken Espresso,80,3,1,45,11,5,5.15,13,0
28,Starbucks,Short Caramel Macchiato,120,5,4,80,15,15,6.15,16,0
26,Starbucks,Birthday Cake Pop,160,2,8,95,16,10,3.25,21,0
19,Starbucks,Small Blonde Vanilla Latte,130,6,3,0,17,15,6.55,18,0
14,Chick-fil-A,8 ct. Nuggets,250,27,11,1210,1,85,7.15,11,0
12,Chick-fil-A,3 ct. Chick-n-Strips,310,29,14,870,2,75,7.49,16,0
4,McDonald's,4 pc. Chicken McNuggets,170,9,10,340,0,25,3.69,10,0
48,Zaxbys,5 Chicken Fingerz,420,45,20,2030,0,90,11.34,16,0


In [ ]:
df.groupby('Restaurant')['Calories '].mean()
# to answer question, which restaurant has the most calories, on average

In [ ]:
#to answer most protein on average
df.groupby('Restaurant')['Protein (g)'].mean()

In [ ]:
df.groupby('Restaurant')['Total Fats (g)'].mean()

In [ ]:
df['Protein_per_Calorie'] = df['Protein (g)'] / df['Calories ']

In [ ]:
df.sort_values(['Protein_per_Calorie'], ascending=False)

In [ ]:
## standardizing sodium, cholestrol, price bc measured in different units. 
from scipy import stats

In [ ]:
df['sodium_mg_z'] = stats.zscore(df['sodium_mg'])
df['cholestrol_z'] = stats.zscore(df['cholestrol_mg'])
df['price_z'] = stats.zscore(df['price'])